In [2]:
import threading
import time
import random

In [3]:
BUFFER_SIZE = 5

# Two buffers: ping (0) and pong (1)
buffer = [[None]*BUFFER_SIZE, [None]*BUFFER_SIZE]

write_buf = 0
read_buf  = 1

data_ready = [False, False]

lock = threading.Lock()

In [4]:
def producer():
    global write_buf, read_buf

    item_id = 0
    while item_id < 10:
        if not data_ready[write_buf]:
            print(f"Producer writing to buffer {write_buf}")

            for i in range(BUFFER_SIZE):
                buffer[write_buf][i] = f"P{item_id}"
                item_id += 1
                time.sleep(0.05)

            data_ready[write_buf] = True

            # swap buffers (NOT ATOMIC)
            write_buf, read_buf = read_buf, write_buf

        time.sleep(0.05)

Debugging

In [5]:
def producer():
    global write_buf, read_buf

    #while True:
    #     print("Producer alive (spin)", flush=True)
    #     time.sleep(0.1)
    item_id = 0
    while item_id < 10:
        if not data_ready[write_buf]:
            print(f"Producer spinning: WB={write_buf}, READY={data_ready}", flush=True)
            time.sleep(0.1)
            print(f"Producer writing to buffer {write_buf}") # Corrected indentation

            for i in range(BUFFER_SIZE):
                buffer[write_buf][i] = f"P{item_id}"
                item_id += 1
                time.sleep(0.05)

            data_ready[write_buf] = True

            # swap buffers (NOT ATOMIC)
            print("Before swap:", write_buf, read_buf, data_ready, flush=True)
            write_buf, read_buf = read_buf, write_buf
            print("After swap:", write_buf, read_buf, data_ready, flush=True)

        time.sleep(0.05)

In [6]:
def consumer():
    global write_buf, read_buf

    consumed = 0
    while consumed < 10:
        if data_ready[read_buf]:
            print(f"Consumer reading from buffer {read_buf}")

            for item in buffer[read_buf]:
                print("   consumed:", item)
                consumed += 1
                time.sleep(0.08)

            data_ready[read_buf] = False

        time.sleep(0.05)

debugging

In [7]:
def consumer():
    global write_buf, read_buf
    #while True:
    #    print("Consumer alive (spin)", flush=True)
    #    time.sleep(0.1)
    consumed = 0
    while consumed < 10:
        if data_ready[read_buf]:
            print(f"Consumer spinning: RB={read_buf}, READY={data_ready}", flush=True)
            time.sleep(0.1)
            print(f"Consumer reading from buffer {read_buf}")

            for item in buffer[read_buf]:
                print("   consumed:", item)
                consumed += 1
                time.sleep(0.08)

            data_ready[read_buf] = False

        time.sleep(0.05)

In [8]:
t1 = threading.Thread(target=producer)
t2 = threading.Thread(target=consumer)

t1.start()
t2.start()

t1.join()
t2.join()

print("Done")

Producer spinning: WB=0, READY=[False, False]
Producer writing to buffer 0
Before swap: 0 1 [True, False]
After swap: 1 0 [True, False]
Producer spinning: WB=1, READY=[True, False]
Consumer spinning: RB=0, READY=[True, False]
Consumer reading from buffer 0
   consumed: P0
Producer writing to buffer 1
   consumed: P1
   consumed: P2
   consumed: P3
Before swap: 1 0 [True, True]
After swap: 0 1 [True, True]
   consumed: P4


KeyboardInterrupt: 